## Dependencies

In [2]:
import pandas as pd


In [3]:
#Load data CSVs to their own dataframes
data_df = pd.read_csv('data/daily.csv')
state_df = pd.read_csv('data/us_state_population.csv')

In [4]:
#Drop the state column that had numbers
state_df = state_df.drop(columns=['state'])

#Make a list of the abbreviations of the states, District of Columbia, and Puerto Rico
state = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','PR']

#Add new state column to df
state_df['state'] = state

#Reorder the columns
state_df = state_df[['state','name','popestimate2019']]

In [6]:
#Save the population data to a new CSV
state_df.to_csv('pop_data.csv', index=False)

In [7]:
data_df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200421,AK,329.0,10790.0,NaN,42.0,36.0,NaN,NaN,NaN,...,36.0,11119.0,11119.0,11119.0,2,0.0,0.0,987.0,8.0,995.0
1,20200421,AL,5231.0,43295.0,NaN,NaN,699.0,NaN,260.0,NaN,...,699.0,48526.0,48526.0,48526.0,1,10.0,58.0,2420.0,206.0,2626.0
2,20200421,AR,2227.0,25214.0,NaN,86.0,291.0,NaN,NaN,27.0,...,291.0,27441.0,27441.0,27441.0,5,1.0,0.0,584.0,304.0,888.0
3,20200421,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200421,AZ,5251.0,49901.0,NaN,637.0,NaN,285.0,NaN,181.0,...,NaN,55152.0,55152.0,55152.0,4,21.0,0.0,465.0,187.0,652.0


In [13]:
#Make a new dataframe using only the columns we want to see
covid_df = data_df[['date','state','positive','total','death']]
covid_df

,date,state,positive,total,death
0,20200421,AK,329.0,11119.0,9.0
1,20200421,AL,5231.0,48526.0,177.0
2,20200421,AR,2227.0,27441.0,43.0
3,20200421,AS,0.0,20.0,NaN
4,20200421,AZ,5251.0,55152.0,208.0
...,...,...,...,...,...
2615,20200301,MI,9.0,9.0,NaN
2616,20200301,RI,1.0,1.0,NaN
2617,20200301,WA,30.0,30.0,8.0
2618,20200229,WA,18.0,18.0,5.0


In [14]:
#Drop the columns for the countries outside of the 50 states and DC
covid_df = covid_df[covid_df.state !='AS']
covid_df = covid_df[covid_df.state !='GU']
covid_df = covid_df[covid_df.state !='MP']
covid_df = covid_df[covid_df.state !='PR']

covid_df

,date,state,positive,total,death
0,20200421,AK,329.0,11119.0,9.0
1,20200421,AL,5231.0,48526.0,177.0
2,20200421,AR,2227.0,27441.0,43.0
4,20200421,AZ,5251.0,55152.0,208.0
5,20200421,CA,33261.0,300100.0,1268.0
...,...,...,...,...,...
2615,20200301,MI,9.0,9.0,NaN
2616,20200301,RI,1.0,1.0,NaN
2617,20200301,WA,30.0,30.0,8.0
2618,20200229,WA,18.0,18.0,5.0


In [15]:
#Save to a new CSV
covid_df.to_csv('covid_data.csv', index=False)

In [ ]:
#Merge tables on state abbreviation
#Add Percent Tested column based on Total/Population
#Add week column 